In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import os

import os
import pandas as pd

carpeta = 'CSVS'  # Asegúrate de reemplazar 'CSVS' con la ruta real de la carpeta

# Obtener la lista de archivos CSV en la carpeta que empiezan por "temp"
archivos_csv = [os.path.join(carpeta, archivo) for archivo in os.listdir(carpeta) if archivo.startswith('temp') and archivo.endswith('.csv')]

# Leer todos los archivos CSV y combinarlos en un solo DataFrame
dfs = []
for archivo_csv in archivos_csv:
    df = pd.read_csv(archivo_csv)
    dfs.append(df)

df_combined = pd.concat(dfs, ignore_index=True)

# Convertir la columna "fase" a minúsculas y capitalizar la primera letra
df_combined['fase'] = df_combined['fase'].astype(str).str.lower().str.capitalize()

# Codificación de variables categóricas
label_encoder = LabelEncoder()
df['equipo_local'] = label_encoder.fit_transform(df['equipo_local'])
df['equipo_visitante'] = label_encoder.transform(df['equipo_visitante'])
df['fase'] = label_encoder.fit_transform(df['fase'])

# Divide los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Define el modelo
class PredictorModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PredictorModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Tamaño de las capas de entrada, oculta y salida
input_size = 3
hidden_size = 5
output_size = 2  # Salida para goles_equipo_local y goles_equipo_visitante

# Instancia del modelo
model = PredictorModel(input_size, hidden_size, output_size)

# Función de pérdida y optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 100  # Puedes ajustar el número de épocas
for epoch in range(num_epochs):
    inputs = torch.tensor(train_data[['fase', 'equipo_local', 'equipo_visitante']].values, dtype=torch.float32)
    labels = torch.tensor(train_data[['goles_equipo_local', 'goles_equipo_visitante']].values, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Guardar el modelo entrenado
torch.save(model.state_dict(), 'modelo_entrenado3.pth')


Epoch [10/100], Loss: 3.7959
Epoch [20/100], Loss: 3.6642
Epoch [30/100], Loss: 3.5541
Epoch [40/100], Loss: 3.4475
Epoch [50/100], Loss: 3.3437
Epoch [60/100], Loss: 3.2407
Epoch [70/100], Loss: 3.1375
Epoch [80/100], Loss: 3.0259
Epoch [90/100], Loss: 2.9100
Epoch [100/100], Loss: 2.7901


In [3]:
import pandas as pd
import torch

# Cargar el modelo entrenado
model = PredictorModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load("modelo_entrenado3.pth"))  # Asegúrate de reemplazar "ruta_del_modelo_entrenado.pth" con la ruta real

# Leer el archivo CSV con los partidos
partidos_df = pd.read_csv("Eliminatoria actual/cuartos.csv")  # Asegúrate de reemplazar "ruta_del_archivo_csv.csv" con la ruta real
# Convertir la columna "fase" a minúsculas y capitalizar la primera letra
partidos_df['fase'] = partidos_df['fase'].str.lower().str.capitalize()

# Crea un diccionario para mapear las etapas a valores numéricos
etapas_dict = {'Grupos': 0, 'Octavos': 1, 'Cuartos': 2, 'Semis': 3, 'Final': 4}

# Reemplaza las etiquetas de las etapas con valores numéricos
partidos_df['fase'] = partidos_df['fase'].map(etapas_dict)

# Crear un DataFrame para almacenar las predicciones completas de goles
predicciones_df = pd.DataFrame(columns=["fase", "equipo_local", "goles_equipo_local", "equipo_visitante", "goles_equipo_visitante"])

# Iterar sobre cada fila del DataFrame de partidos
for index, row in partidos_df.iterrows():
    # Obtener los datos de entrada para hacer la predicción
    fase = label_encoder.transform([row["fase"]])[0]  # Transformar la fase a su valor codificado
    equipo_local = row["equipo_local"]
    equipo_visitante = row["equipo_visitante"]
    datos_prediccion = torch.tensor([[fase, equipo_local, equipo_visitante]], dtype=torch.float32)
    
    # Realizar la predicción utilizando el modelo
    predicciones = model(datos_prediccion).detach().numpy().round().astype(int)
    
    # Crear un nuevo DataFrame con la predicción completa para este partido
    nueva_fila = {
        "fase": row["fase"],
        "equipo_local": row["equipo_local"],
        "goles_equipo_local": predicciones[0][0],
        "equipo_visitante": row["equipo_visitante"],
        "goles_equipo_visitante": predicciones[0][1]
    }
    predicciones_df = predicciones_df.append(nueva_fila, ignore_index=True)


# Guardar el DataFrame con las predicciones completas en un nuevo archivo CSV
predicciones_df.to_csv("predicciones_completas.csv", index=False)


ValueError: y contains previously unseen labels: 2

In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os

# Definir el modelo PredictorModel
class PredictorModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PredictorModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Ruta de la carpeta que contiene los archivos CSV
carpeta = 'CSVS'

# Obtener la lista de archivos CSV en la carpeta que empiezan por "temp"
archivos_csv = [os.path.join(carpeta, archivo) for archivo in os.listdir(carpeta) if archivo.startswith('temp') and archivo.endswith('.csv')]

# Leer todos los archivos CSV y combinarlos en un solo DataFrame
dfs = []
for archivo_csv in archivos_csv:
    df = pd.read_csv(archivo_csv)
    dfs.append(df)

df_combined = pd.concat(dfs, ignore_index=True)

# Convertir la columna "fase" a minúsculas y capitalizar la primera letra
df_combined['fase'] = df_combined['fase'].str.lower().str.capitalize()

# Codificación de variables categóricas
label_encoder = LabelEncoder()
df_combined['fase'] = label_encoder.fit_transform(df_combined['fase'])
df_combined['equipo_local'] = label_encoder.fit_transform(df_combined['equipo_local'])
df_combined['equipo_visitante'] = label_encoder.fit_transform(df_combined['equipo_visitante'])

# Divide los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(df_combined, test_size=0.2, random_state=42)

# Tamaño de las capas de entrada, oculta y salida
input_size = 3
hidden_size = 5
output_size = 2

# Instancia del modelo PredictorModel
model = PredictorModel(input_size, hidden_size, output_size)

# Función de pérdida y optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 100
for epoch in range(num_epochs):
    inputs = torch.tensor(train_data[['fase', 'equipo_local', 'equipo_visitante']].values, dtype=torch.float32)
    labels = torch.tensor(train_data[['goles_equipo_local', 'goles_equipo_visitante']].values, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Guardar el modelo entrenado
torch.save(model.state_dict(), 'modelo_entrenado.pth')

# Cargar el modelo entrenado
model = PredictorModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load("modelo_entrenado.pth"))

# Leer el archivo CSV con los partidos
partidos_df = pd.read_csv("Eliminatoria actual/cuartos.csv")

# Convertir la columna "fase" a minúsculas y capitalizar la primera letra
partidos_df['fase'] = partidos_df['fase'].str.lower().str.capitalize()

# Reemplazar las etiquetas de las etapas con valores numéricos
partidos_df['fase'] = label_encoder.transform(partidos_df['fase'])

# Crear un DataFrame para almacenar las predicciones completas de goles
predicciones_df = pd.DataFrame(columns=["fase", "equipo_local", "goles_equipo_local", "equipo_visitante", "goles_equipo_visitante"])

# Iterar sobre cada fila del DataFrame de partidos
for index, row in partidos_df.iterrows():
    # Obtener los datos de entrada para hacer la predicción
    fase = row["fase"]
    equipo_local = row["equipo_local"]
    equipo_visitante = row["equipo_visitante"]
    datos_prediccion = torch.tensor([[fase, equipo_local, equipo_visitante]], dtype=torch.float32)
    
    # Realizar la predicción utilizando el modelo
    predicciones = model(datos_prediccion).detach().numpy().round().astype(int)
    
    # Crear un nuevo DataFrame con la predicción completa para este partido
    nueva_fila = {
        "fase": row["fase"],
        "equipo_local": row["equipo_local"],
        "goles_equipo_local": predicciones[0][0],
        "equipo_visitante": row["equipo_visitante"],
        "goles_equipo_visitante": predicciones[0][1]
    }
    predicciones_df = predicciones_df.append(nueva_fila, ignore_index=True)

# Guardar el DataFrame con las predicciones completas en un nuevo archivo CSV
predicciones_df.to_csv("predicciones_completas.csv", index=False)


Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [40/100], Loss: nan
Epoch [50/100], Loss: nan
Epoch [60/100], Loss: nan
Epoch [70/100], Loss: nan
Epoch [80/100], Loss: nan
Epoch [90/100], Loss: nan
Epoch [100/100], Loss: nan


ValueError: y contains previously unseen labels: 'Cuartos'